In [ ]:
import tensorflow as tf
import dlib
import cv2
import numpy as np
import os
import ipywidgets as widgets
from io import BytesIO
from colorthief import ColorThief
import matplotlib.pyplot as plt

In [ ]:
# This file used for integrating shape predictor(eye tracker) and jaundice detection

# Load the Dlib shape predictor
shape_predictor = dlib.shape_predictor("models/shape_predictor_68_face_landmarks.dat")

# Load your jaundice detection model
jaundice_model = tf.keras.models.load_model("models/jaundice_detector.keras")

In [34]:
def full_face_eye_cropping(img, M = 36, N = 42):
    """
    
    This is the function to crop and save the eye for an input of a full face
    
    Input:
    img = The input image loaded with dlib
    M, N = The landmark region of intrest ((42,46) for left eye and (36,42) for right eye)
    
    Output:
    eye = The cropped image of the input where only the eye is present in the size of (150,150)
    
    """
    
    # Defining Pre-Trained Models
    face_detector = dlib.get_frontal_face_detector()
    landmark_detector = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    
    # Detecting the faces in the input
    faces = face_detector(img, 1)
    face = faces[0]
    
    # Detecting landmarks in the image
    base_img = img.copy()
    landmarks_tuple = []
    landmarks = landmark_detector(img,face)

    for i in range(0, 68):
        x = landmarks.part(i).x
        y = landmarks.part(i).y

        landmarks_tuple.append((x, y))
            
        cv2.circle(base_img, (x, y), 2, (255, 255, 255), -1)
    
    # Creating a route between landmarks
    routes = [i for i in range(M,N)] + [M]
    route_coordinates = []
    base_img = img.copy()

    # Filling route_coordinates
    for i in range(0, len(routes) - 1):
        source_point = routes[i]
        target_point = routes[i+1]
    
        source_coordinate = landmarks_tuple[source_point]
        target_coordinate = landmarks_tuple[target_point]
    
        route_coordinates.append(source_coordinate)
    
        cv2.line(base_img, source_coordinate, target_coordinate, (255, 255, 255), 2)
        
    # Getting the eye bounding box using route_coordinates
    x_coords = [coord[0] for coord in route_coordinates]
    y_coords = [coord[1] for coord in route_coordinates]
    x_min, x_max = min(x_coords), max(x_coords)
    y_min, y_max = min(y_coords), max(y_coords)

    # Add padding
    padding = 10
    x_min = max(0, x_min - padding)
    y_min = max(0, y_min - padding)
    x_max = min(img.shape[1], x_max + padding)
    y_max = min(img.shape[0], y_max + padding)

    # Crop and resize
    eye_region = img[y_min:y_max, x_min:x_max]
    eye = cv2.resize(eye_region, (150, 150))
    
    return eye

In [ ]:
# Creating custom preprocessing layer

class PreprocessingLayer(tf.keras.layers.Layer):
    def __init__(self, target_size=(100, 100)):
        super(PreprocessingLayer, self).__init__()
        self.target_size = target_size

    def call(self, inputs):
        # Unpack inputs
        image_input, color_data_input = inputs

        # Preprocessing the image: resizing
        processed_image = tf.image.resize(image_input, self.target_size)

        # Preprocessing the color data: normalize to range [0, 1]
        processed_color_data = color_data_input / 255.0

        return processed_image, processed_color_data

    def compute_output_shape(self, input_shape):
        # Output shapes for both inputs
        image_shape, color_data_shape = input_shape
        return (
            (image_shape[0], self.target_size[0], self.target_size[1], image_shape[3]),
            (color_data_shape[0], color_data_shape[1]),
        )

In [36]:
# Define the integrated model
inputs = tf.keras.Input(shape=(None, None, 3), name='image_input')
color_data_input = tf.keras.Input(shape=(3,), name='color_data_input')
x_image, x_color = PreprocessingLayer()([inputs, color_data_input])  # Preprocess both inputs
outputs = jaundice_model([x_image, x_color])
integrated_model = tf.keras.Model([inputs, color_data_input], outputs)
integrated_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ color_data_input    │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ preprocessing_laye… │ [(None, 100, 100, │          0 │ image_input[0][0… │
│ (PreprocessingLaye… │ 3), (None, 3)]    │            │ color_data_input… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional          │ (None, 1)         │    912,769 │ preprocessing_la… │
│ (Functional)        │                   │            │ preprocessing_la… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 912,769 (3.48 MB)

 Trainable params: 912,769 (3.48 MB)

 Non-trainable params: 0 (0.00 B)

In [43]:
# Uploader to check integrated model by uploading an image

uploaded = widgets.FileUpload(accept="image/*", multiple=True)
display(uploaded)
out = widgets.Output()
display(out)

image_size = 100  

temp_dir = "temp_files"
os.makedirs(temp_dir, exist_ok=True) 

def process_uploaded_files(change):
    """Proses file yang diunggah ketika terjadi perubahan di widget."""
    with out:
        out.clear_output()

        for file_data in uploaded.value:
            file_name = file_data['name']
            file_content = file_data['content']
            
            print(f'User uploaded file "{file_name}" with length {len(file_content)} bytes')
            try:

                temp_file_path = os.path.join(temp_dir, file_name)
                with open(temp_file_path, "wb") as temp_file:
                    temp_file.write(file_content)

  
                file_bytes = np.frombuffer(file_content, np.uint8)
                img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
                # img = cv2.resize(img, (image_size, image_size))

  
                color_thief = ColorThief(temp_file_path) 
                dominant_color = color_thief.get_color(quality=1)
                normalized_color = [float(c) / 255.0 for c in dominant_color]

               
                img = np.expand_dims(img, axis=0) / 255.0

               
                prediction = integrated_model.predict([img, np.array([normalized_color])])

                if prediction[0][0] > 0.5:
                    print(f"The image {file_name} is classified as Normal Eye")
                else:
                    print(f"The image {file_name} is classified as Jaundiced Eye")

              
                try:
                   
                    img_display = (img[0] * 255).astype(np.uint8) 
                    plt.imshow(cv2.cvtColor(img_display, cv2.COLOR_BGR2RGB))  
                    plt.axis('off')
                    plt.show()
                except Exception as e:
                    print(f"Error displaying image: {e}")

            except Exception as e:
                print(f"Error processing {file_name}: {e}")

            finally:
              
                if os.path.exists(temp_file_path):
                    os.remove(temp_file_path)


uploaded.observe(process_uploaded_files, names='value')


FileUpload(value=(), accept='image/*', description='Upload', multiple=True)

Output()

In [38]:
# Save the integrated model to keras format
integrated_model.save("integrated_model.keras")

In [39]:
# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(integrated_model)
tflite_model = converter.convert()

# Save the converted model to a file
with open('final_model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\muham\AppData\Local\Temp\tmpe2n49j1e\assets


INFO:tensorflow:Assets written to: C:\Users\muham\AppData\Local\Temp\tmpe2n49j1e\assets


Saved artifact at 'C:\Users\muham\AppData\Local\Temp\tmpe2n49j1e'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): List[TensorSpec(shape=(None, None, None, 3), dtype=tf.float32, name='image_input'), TensorSpec(shape=(None, 3), dtype=tf.float32, name='color_data_input')]
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2324121529760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2324121780640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2324121784688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2324121785216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2324121790144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2324121791024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2324121794720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2324121815696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2324121819920: TensorSpec(shape=(), dtype=tf.resource, name=No

In [40]:
import pickle

# Save the model object as a .pkl file
with open('final_model.pkl', 'wb') as f:
    pickle.dump(integrated_model, f)


In [41]:
import json

# Save the architecture (structure) to a .json file
model_json = integrated_model.to_json()  # Get the architecture as a JSON string

# Write the JSON string to a file
with open('final_model.json', 'w') as json_file:
    json.dump(model_json, json_file)